In [2]:
"""
prerequisites for step one and two of implementing reading vector:
- need to have pairs of size 5 to 128
- determine the hidden state values with respect to the chosen LAT token position (which is [-2] for us) + probably pick a layer we want
- compute differences between hidden states within each pair, which is already done
- we normalize the difference -> should be [128, 4096], which 128 being the sample pairs and 4096 being the activation vector size
"""

'\nprerequisites for step one and two of implementing reading vector:\n- need to have pairs of size 5 to 128\n- determine the hidden state values with respect to the chosen LAT token position (which is [-2] for us) + probably pick a layer we want\n- compute differences between hidden states within each pair, which is already done\n- we normalize the difference -> should be [128, 4096], which 128 being the sample pairs and 4096 being the activation vector size\n'

In [3]:
import numpy as np
import torch

In [4]:
data_np = np.random.rand(100,4096)
data_tensor = torch.tensor(data_np)
print(data_tensor)
print(data_tensor.shape)

tensor([[0.1991, 0.0471, 0.0378,  ..., 0.4061, 0.0889, 0.4647],
        [0.9467, 0.5530, 0.7832,  ..., 0.0520, 0.8006, 0.4639],
        [0.9299, 0.2906, 0.7704,  ..., 0.4612, 0.8418, 0.2608],
        ...,
        [0.7174, 0.5835, 0.8745,  ..., 0.6223, 0.5199, 0.2843],
        [0.7495, 0.5361, 0.0674,  ..., 0.2944, 0.7182, 0.3075],
        [0.9583, 0.3821, 0.1215,  ..., 0.2201, 0.0928, 0.9772]],
       dtype=torch.float64)
torch.Size([100, 4096])


In [5]:
# normalize
mean = torch.mean(data_tensor)
std = torch.std(data_tensor)
data_normalized = (data_tensor-mean)/std

In [6]:
data_tensor = data_tensor - data_tensor.mean() # center the data
cov_matrix = torch.mm(data_tensor.t(), data_tensor) / data_tensor[0]-1 # 100-1

In [35]:
eigenvalues, eigenvectors = np.linalg.eig(np.array(cov_matrix))

In [70]:
# print(eigenvalues[0])
print(np.max(eigenvalues))
# print(eigenvectors[0])

(56142.04512951622+0j)


In [63]:
zipped = zip(eigenvalues, eigenvectors)
# lst_zipped = list(zipped)

In [64]:
res = sorted(zipped, reverse=True)

In [65]:
eigenvalues, eigenvectors = zip(*res)

In [66]:
print(eigenvalues[0])
print(eigenvectors[0])

(56142.04512951622+0j)
[-0.0056678 +0.j  0.00587951+0.j -0.00903555+0.j ... -0.01162932+0.j
 -0.01172248+0.j -0.02868854+0.j]


In [67]:
data_pca = torch.pca_lowrank(data_normalized, center=True) # no need to center because its normalized
print(len(data_pca))
print(data_pca[0].shape) # U
print(data_pca[1].shape) # S
# S**2/(samplesize-1) contains eigenvectors of ATA/(samplesize-1) which is the covariance of A when center=True is provided
print(data_pca[2].shape) # V columns represent the principal directions

first_pca = data_pca[2][:, :1] # gets the first principal component
print(first_pca, first_pca.shape)
first_pca = first_pca.squeeze()
print(first_pca, first_pca.shape)

3
torch.Size([100, 6])
torch.Size([6])
torch.Size([4096, 6])
tensor([[ 0.0179],
        [ 0.0131],
        [-0.0043],
        ...,
        [ 0.0105],
        [-0.0141],
        [-0.0155]], dtype=torch.float64) torch.Size([4096, 1])
tensor([ 0.0179,  0.0131, -0.0043,  ...,  0.0105, -0.0141, -0.0155],
       dtype=torch.float64) torch.Size([4096])


In [68]:
# then you apply the first PCA on the same stimuli set S to obtain scores because
# "in practice, the 'reading vector' v is also multiplied by a 'sign' component"
scores = torch.tensor(np.dot(np.array(data_normalized), np.array(first_pca))) # [100,4096] T [4096]
# each score should be one-dimensional, so dot product btwn normalized and first pca is needed
big_dim_scores = data_normalized*first_pca
print(scores.shape)
print(big_dim_scores)
print()
# each stimulus has one score, which is why the shape of scores is torch.Size([100])
updated_vectors = first_pca*scores[0] # one stimulus (in the loop, change the index based on the iteration you are on)
print(updated_vectors.shape)

torch.Size([100])
tensor([[-0.0186, -0.0205,  0.0068,  ..., -0.0034,  0.0200,  0.0018],
        [ 0.0278,  0.0024, -0.0042,  ..., -0.0162, -0.0147,  0.0019],
        [ 0.0267, -0.0094, -0.0040,  ..., -0.0014, -0.0167,  0.0128],
        ...,
        [ 0.0135,  0.0038, -0.0055,  ...,  0.0045, -0.0010,  0.0115],
        [ 0.0155,  0.0017,  0.0064,  ..., -0.0074, -0.0107,  0.0103],
        [ 0.0285, -0.0053,  0.0056,  ..., -0.0101,  0.0198, -0.0257]],
       dtype=torch.float64)

torch.Size([4096])


In [55]:
updated_reading_vectors = scores.detach().clone()

for i in range(len(scores)):
    updated_reading_vectors[i] *= scores[i]

Inference

In [56]:
# find the hidden states at the token position [-2]
# these are normalized uding the parameters derived using the construction of the PCA model in the training phase (mean and std variables from before)
# calculate the dot product between the normalized hidden states and out reading vector, which yields a set of scores

data_inference = np.random.rand(10,4096)
data_inference_tensor = torch.tensor(data_np)
data_inference_normalized = (data_inference_tensor-mean)/std
data_inference_pca = torch.pca_lowrank(data_inference_normalized) # no need to center because its normalized

# first_pca_inference = data_inference_pca[2][:, :1] # gets the first principal component
# first_pca_inference = first_pca_inference.squeeze()
# print(first_pca_inference, first_pca_inference.shape)

scores = data_inference_normalized*first_pca
print(scores, scores.shape)

tensor([[-0.0106,  0.0051, -0.0340,  ...,  0.0030,  0.0175,  0.0054],
        [ 0.0158, -0.0006,  0.0209,  ...,  0.0146, -0.0128,  0.0055],
        [ 0.0152,  0.0024,  0.0200,  ...,  0.0012, -0.0146,  0.0372],
        ...,
        [ 0.0077, -0.0010,  0.0277,  ..., -0.0040, -0.0009,  0.0335],
        [ 0.0088, -0.0004, -0.0318,  ...,  0.0067, -0.0093,  0.0299],
        [ 0.0162,  0.0013, -0.0278,  ...,  0.0091,  0.0173, -0.0747]],
       dtype=torch.float64) torch.Size([100, 4096])
